# Compressed Sensing Simulation

## Getting Started

In [1]:
import h5py
import numpy as np
import pandas as pd
import scanpy as sc
import spams
from scipy.stats import entropy
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
from scipy.io import mmread
from scipy.stats import spearmanr, pearsonr, entropy
from scipy.spatial import distance
import os

THREADS = 10

In [4]:
# Define file paths
matrix_file = "dataset/GSE165371_cb_adult_mouse/cb_adult_mouse.mtx.gz"
barcodes_file = "dataset/GSE165371_cb_adult_mouse/cb_adult_mouse_barcodes.txt"
genes_file = "dataset/GSE165371_cb_adult_mouse/cb_adult_mouse_genes.txt"

# Load the data
matrix = mmread(matrix_file).tocsc()  # Read the matrix market file
barcodes = pd.read_csv(barcodes_file, header=None).iloc[:, 0].tolist()  # Read barcodes
genes = pd.read_csv(genes_file, header=None).iloc[:, 0].tolist()  # Read genes


print(matrix.shape)
print(len(barcodes))  # Number of barcodes
print(len(genes))     # Number of genes


# AnnData creation
adata = sc.AnnData(
    X=matrix.T,  # Transpose the matrix to match AnnData's format
    obs=pd.DataFrame(index=barcodes),  # Barcodes -> obs
    var=pd.DataFrame(index=genes)      # Genes -> var
)

print(adata)

# Save the AnnData object to an H5AD file (optional)
adata.write("dataset/cerebellum/cb_adult_mouse.h5ad")

# verify
print(adata.var)
print(adata.obs)

(24409, 611034)
611034
24409
AnnData object with n_obs × n_vars = 611034 × 24409
Empty DataFrame
Columns: []
Index: [Xkr4, Gm1992, Gm37381, Rp1, Sox17, Mrpl15, Lypla1, Gm37988, Tcea1, Rgs20, Atp6v1h, Rb1cc1, 4732440D04Rik, Fam150a, St18, Pcmtd1, Gm26901, Sntg1, Rrs1, Adhfe1, Mybl1, Vcpip1, 1700034P13Rik, Sgk3, Mcmdc2, Snhg6, Tcf24, Ppp1r42, Gm15818, Cops5, Cspp1, Arfgef1, Cpa6, Prex2, A830018L16Rik, Sulf1, Slco5a1, Gm29283, Prdm14, Ncoa2, Gm29570, Tram1, Lactb2, Eya1, Trpa1, Kcnb2, Terf1, Sbspon, 4930444P10Rik, Rpl7, Rdh10, Gm28095, Stau2, Gm7568, Ube2w, Tceb1, D030040B21Rik, Tmem70, Ly96, Gm28376, Jph1, Gdap1, Pi15, Gm28154, Gm16070, Crispld1, Gm28153, Defb41, Tfap2d, Tfap2b, Pkhd1, Il17f, Mcm3, 6720483E21Rik, Paqr8, Efhc1, Tram2, Tmem14a, Gsta3, Gm28836, Kcnq5, Rims1, Gm29107, Ogfrl1, Gm28822, B3gat2, Smap1, Sdhaf4, Fam135a, Col9a1, Col19a1, Lmbrd1, Adgrb3, Phf3, Ptp4a1, Gm29669, Lgsn, Khdrbs2, Prim2, Rab23, ...]

[24409 rows x 0 columns]
Empty DataFrame
Columns: []
Index: [IXa_M003_

In [2]:
adata = sc.read_h5ad("dataset/cerebellum/cb_adult_mouse.h5ad")

In [3]:
def downsample_adata(adata, gene_set_size, cell_count=10000, output_path="dataset/cerebellum/cb_adult_mouse"):
    """
    Downsamples an AnnData object based on a specified gene set size.
    
    Parameters:
    - adata: AnnData object to be downsampled
    - gene_set_size: int, number of genes to include (500, 1000, or 5000)
    - cell_count: int, number of cells to select (default: 10,000)
    - output_path: str, base path for saving the downsampled file
    
    Returns:
    - AnnData object with selected genes and cells
    """
    assert gene_set_size in [500, 1000, 5000], "Gene set size must be 500, 1000, or 5000."
    
    # Load genes from file
    gene_file = f"./dataset/genes_{gene_set_size}.csv"
    genes = pd.read_csv(gene_file, header=None)[1].tolist()
    
    # Normalize gene names
    adata.var_names = adata.var_names.str.upper().str.strip()
    genes = [gene.upper().strip() for gene in genes]
    
    # Match genes with existing ones in the dataset
    selected_genes = [gene for gene in genes if gene in adata.var_names]
    print(f"Matched genes for {gene_set_size} set: {len(selected_genes)}")
    
    # Ensure unique gene names
    adata.var_names_make_unique()
    
    # Randomly select cells
    np.random.seed(23)
    selected_cells = np.random.choice(adata.obs_names, size=cell_count, replace=False)
    
    # Subset the AnnData object
    adata_subset = adata[selected_cells, selected_genes]
    
    # Save the new dataset
    output_file = f"{output_path}_{gene_set_size}genes.h5ad"
    adata_subset.write(output_file)
    print(f"Downsampled dataset saved to {output_file}")
    
    return adata_subset

In [4]:
def split_and_save_data(adata, dataset_prefix, output_dir="./dataset/"):
    """
    Splits `adata.X` into train, validation, and test subsets and saves them in the specified directory.
    
    Parameters:
    - adata: AnnData object, containing the dataset.
    - dataset_prefix: str, manually specified prefix for saved dataset files.
    - output_dir: str, directory to save the subsets.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Ensure `adata.X` is in a compatible format
    X = adata.X
    
    # Convert sparse matrices to compressed row format
    if sp.issparse(X):  
        X = X.tocsr()
    else:
        X = np.asarray(X, dtype=np.float64)
    
    # Get total number of samples (cells)
    num_cells = X.shape[0]
    
    # Define split sizes (50% Train, 25% Validate, 25% Test)
    train_idx, temp_idx = train_test_split(np.arange(num_cells), test_size=0.50, random_state=23)
    validate_idx, test_idx = train_test_split(temp_idx, test_size=0.50, random_state=23)
    
    # Extract data using indices
    train_data = X[train_idx]
    validate_data = X[validate_idx]
    test_data = X[test_idx]
    
    # Convert all subsets to dense arrays before saving
    train_data = np.asarray(train_data.todense() if sp.issparse(train_data) else train_data, dtype=np.float64)
    validate_data = np.asarray(validate_data.todense() if sp.issparse(validate_data) else validate_data, dtype=np.float64)
    test_data = np.asarray(test_data.todense() if sp.issparse(test_data) else test_data, dtype=np.float64)

    # Convert train_data to Fortran-contiguous format for SPAMS
    train_data = np.asfortranarray(train_data)
    
    # Save subsets with manually inputted prefix
    np.save(os.path.join(output_dir, f"{dataset_prefix}_train_data.npy"), train_data)
    np.save(os.path.join(output_dir, f"{dataset_prefix}_validate_data.npy"), validate_data)
    np.save(os.path.join(output_dir, f"{dataset_prefix}_test_data.npy"), test_data)
    
    # Print dataset shapes
    print(f"Train data shape: {train_data.shape}")
    print(f"Validation data shape: {validate_data.shape}")
    print(f"Test data shape: {test_data.shape}")
    print(f"Datasets saved in {output_dir} as {dataset_prefix}_train_data.npy, {dataset_prefix}_validate_data.npy, and {dataset_prefix}_test_data.npy")

In [5]:
def load_saved_data(dataset_name, input_dir="./dataset/"):
    """
    Loads the train, validation, and test datasets from the specified directory using the given dataset name.
    
    Parameters:
    - dataset_name: str, name of the dataset (prefix used when saving)
    - input_dir: str, directory containing the saved subsets
    
    Returns:
    - train_data: numpy array
    - validate_data: numpy array
    - test_data: numpy array
    """
    if not os.path.exists(input_dir):
        raise FileNotFoundError(f"Directory '{input_dir}' does not exist.")

    # Construct file paths using the dataset name
    train_data_path = os.path.join(input_dir, f"{dataset_name}_train_data.npy")
    validate_data_path = os.path.join(input_dir, f"{dataset_name}_validate_data.npy")
    test_data_path = os.path.join(input_dir, f"{dataset_name}_test_data.npy")

    # Check if files exist before loading
    for file_path in [train_data_path, validate_data_path, test_data_path]:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")

    # Load data
    train_data = np.load(train_data_path, allow_pickle=False)
    validate_data = np.load(validate_data_path, allow_pickle=False)
    test_data = np.load(test_data_path, allow_pickle=False)

    # Ensure all loaded data is in dense format
    validate_data = np.asarray(validate_data, dtype=np.float64)
    test_data = np.asarray(test_data, dtype=np.float64)
    
    train_data, validate_data, test_data = train_data.T, validate_data.T, test_data.T

    # Print dataset shapes
    print(f"Loaded dataset: {dataset_name}")
    print(f"Train data shape: {train_data.shape}")
    print(f"Validation data shape: {validate_data.shape}")
    print(f"Test data shape: {test_data.shape}")
    
    return train_data, validate_data, test_data

## Utilities

In [6]:
def random_phi_subsets_g(m, g, n, d_thresh=0.4):
    """
    Generates a random measurement matrix (Phi) with m pools and g genes while ensuring minimal correlation.
    
    Parameters:
        m (int): Number of pools (samples).
        g (int): Number of genes.
        n (tuple): Range (min, max) for the number of pools each gene is assigned to.
        d_thresh (float): Maximum allowable correlation threshold between gene assignments.
    
    Returns:
        np.ndarray: A (m x g) matrix where each column represents gene assignment to pools.
    """
    Phi = np.zeros((m, g))  # Initialize measurement matrix
    Phi[np.random.choice(m, np.random.randint(n[0], n[1]+1), replace=False), 0] = 1  # Assign first gene
    
    for i in range(1, g):  # Assign pools for remaining genes
        dmax = 1
        while dmax > d_thresh:
            p = np.zeros(m)
            p[np.random.choice(m, np.random.randint(n[0], n[1]+1), replace=False)] = 1  # Randomly assign pools
            dmax = 1 - distance.cdist(Phi[:, :i].T, [p], 'correlation').min()  # Ensure minimal correlation
        
        Phi[:, i] = p  # Assign gene to pools
    
    Phi = (Phi.T / Phi.sum(1)).T  # Normalize by row sum
    return Phi

def get_observations(X0, Phi, snr=5, return_noise=False):
    """
    Simulates observations by applying the measurement matrix (Phi) to the true signal (X0) with added noise.
    
    Parameters:
        X0 (np.ndarray): Original gene expression matrix (genes x samples).
        Phi (np.ndarray): Measurement matrix mapping genes to pools.
        snr (float): Signal-to-noise ratio.
        return_noise (bool): If True, return noise separately.
    
    Returns:
        np.ndarray: Noisy observed measurements (pools x samples).
        (Optional) np.ndarray: Noise matrix.
    """
    noise = np.array([np.random.randn(X0.shape[1]) for _ in range(X0.shape[0])])
    noise *= np.linalg.norm(X0) / np.linalg.norm(noise) / snr  # Scale noise level
    
    if return_noise:
        return Phi.dot(X0 + noise), noise
    else:
        return Phi.dot(X0 + noise)

def compare_distances(A, B, random_samples=[], s=200, pvalues=False):
    """
    Compares the Euclidean distances between corresponding columns of matrices A and B.
    
    Parameters:
        A (np.ndarray): First data matrix.
        B (np.ndarray): Second data matrix.
        random_samples (list, optional): Boolean mask of selected samples.
        s (int): Number of samples to randomly select if not provided.
        pvalues (bool): Whether to return p-values for the correlation tests.
    
    Returns:
        tuple: Pearson and Spearman correlation coefficients (and p-values if pvalues=True).
    """
    if len(random_samples) == 0:
        random_samples = np.zeros(A.shape[1], dtype=bool)
        random_samples[:min(s, A.shape[1])] = True  # Select `s` random samples
        np.random.shuffle(random_samples)
    
    dist_x = distance.pdist(A[:, random_samples].T, 'euclidean')
    dist_y = distance.pdist(B[:, random_samples].T, 'euclidean')
    
    pear = pearsonr(dist_x, dist_y)  # Pearson correlation
    spear = spearmanr(dist_x, dist_y)  # Spearman correlation
    
    if pvalues:
        return pear, spear  # Return correlations with p-values
    else:
        return pear[0], spear[0]  # Return only correlation coefficients

def correlations(A, B):
    """
    Computes correlation metrics between two matrices A and B.
    
    Parameters:
        A (np.ndarray): First matrix.
        B (np.ndarray): Second matrix.
    
    Returns:
        tuple: Overall correlation, Spearman correlation, per-gene correlation, per-sample correlation.
    """
    p = (1 - distance.correlation(A.flatten(), B.flatten()))  # Overall correlation
    spear = spearmanr(A.flatten(), B.flatten())  # Spearman correlation
    
    dist_genes = np.zeros(A.shape[0])  # Per-gene correlation
    for i in range(A.shape[0]):
        dist_genes[i] = 1 - distance.correlation(A[i], B[i])
    
    pg = np.average(dist_genes[np.isfinite(dist_genes)])  # Mean per-gene correlation
    
    dist_sample = np.zeros(A.shape[1])  # Per-sample correlation
    for i in range(A.shape[1]):
        dist_sample[i] = 1 - distance.correlation(A[:, i], B[:, i])
    
    ps = np.average(dist_sample[np.isfinite(dist_sample)])  # Mean per-sample correlation
    
    return p, spear[0], pg, ps

def compare_results(A, B):
    """
    Aggregates multiple comparison metrics between matrices A and B.
    
    Parameters:
        A (np.ndarray): First matrix.
        B (np.ndarray): Second matrix.
    
    Returns:
        list: Combined correlation and distance comparison results.
    """
    results = list(correlations(A, B))[:-1]  # Get correlation metrics (excluding `ps`)
    results += list(compare_distances(A, B))  # Compare Euclidean distances (gene-wise)
    results += list(compare_distances(A.T, B.T))  # Compare Euclidean distances (sample-wise)
    return results

## Sparse Decoding

In [7]:
def sparse_decode(Y, D, k, worstFit=1., mink=0, method='omp', nonneg=False):
    """
    Sparse decoding method - obtain module activations from composite measurements.
    
    Parameters:
    - Y : Observed measurement data (pools × samples)
    - D : Dictionary (module patterns) used for reconstruction
    - k : Sparsity constraint (number of nonzero coefficients)
    - worstFit : Minimum required reconstruction accuracy (default = 1)
    - mink : Minimum sparsity allowed (default = 0)
    - method : 'omp' (Orthogonal Matching Pursuit) or 'lasso' (L1 regularization)
    - nonneg : If True, forces non-negative solutions (for LASSO only)
    
    Returns:
    - W : Estimated module activations (modules × samples)
    """
    
    if method == 'omp':
        while k > mink:
            W = spams.omp(np.asfortranarray(Y), np.asfortranarray(D), L=k, numThreads=THREADS)
            W = np.asarray(W.todense())

            fit = 1 - np.linalg.norm(Y - D.dot(W))**2 / np.linalg.norm(Y)**2  # Compute fit accuracy

            if fit < worstFit:
                break  # Stop if fit is too low
            else:
                k -= 1  # Decrease sparsity constraint
                
            
    elif method == 'lasso':
        Ynorm = np.linalg.norm(Y)**2 / Y.shape[1]  # Normalize Y
        W = spams.lasso(np.asfortranarray(Y), np.asfortranarray(D),
                        lambda1=k * Ynorm, mode=1, numThreads=THREADS, pos=nonneg)
        W = np.asarray(W.todense())
    
    return W

## Modules Dictionary

In [8]:
def smaf(X, d, lda1, lda2, maxItr=10, UW=None, posW=False, posU=True, use_chol=False, 
         module_lower=1, activity_lower=1, donorm=False, mode=1, mink=5, U0=[], 
         U0_delta=0.1, doprint=False):
    
    """
    Sparse Module Activity Factorization (SMAF) to learn gene modules.
    
    Parameters:
    - X: np.array, shape (genes, cells), gene expression matrix. Note: dims opposite of anndata 
    - d: int, number of gene modules (dictionary size)
    - lda1: float, regularization term for activity sparsity
    - lda2: float, regularization term for module sparsity
    - maxItr: int, number of iterations for optimization
    - UW: tuple, (U, W) initial matrices, optional
    - posW: bool, enforce non-negativity on W
    - posU: bool, enforce non-negativity on U
    - use_chol: bool, use Cholesky decomposition for faster computation
    - module_lower: float, lower bound for module entropy
    - activity_lower: float, lower bound for activity entropy
    - donorm: bool, normalize U matrix
    - mode: int, optimization mode
    - mink: int, minimum sparsity constraint
    - U0: list, optional initialization for U
    - U0_delta: float, delta for projected gradient descent
    - doprint: bool, print progress
    
    Returns:
    - U, W: np.arrays, learned dictionary and activity matrices
    """
    # Initialize U and W matrices if not provided
    if UW is None:
        U, W = spams.nmf(np.asfortranarray(X), return_lasso=True, K=d, numThreads=THREADS)
        W = np.asarray(W.todense())  # Convert sparse W to dense format
    else:
        U, W = UW  # Use provided matrices
    
    # Compute initial reconstruction of X
    Xhat = U.dot(W)
    # Compute normalization factor for regularization
    Xnorm = np.linalg.norm(X) ** 2 / X.shape[1]
    
    # Iterate to optimize U and W
    for itr in range(maxItr):
        if mode == 1:
            # Solve for U using Lasso regression with sparsity regularization
            U = spams.lasso(np.asfortranarray(X.T), D=np.asfortranarray(W.T),
                            lambda1=lda2 * Xnorm, mode=1, numThreads=THREADS,
                            cholesky=use_chol, pos=posU)
            U = np.asarray(U.todense()).T  # Convert to dense and transpose
        elif mode == 2:
            # Optionally use projected gradient descent if U0 is provided
            if len(U0) > 0:
                U = projected_grad_desc(W.T, X.T, U.T, U0.T, lda2, U0_delta, maxItr=400)
                U = U.T  # Transpose back
            else:
                # Solve for U using Lasso regression with different lambda settings
                U = spams.lasso(np.asfortranarray(X.T), D=np.asfortranarray(W.T),
                                lambda1=lda2, lambda2=0.0, mode=2, numThreads=THREADS,
                                cholesky=use_chol, pos=posU)
                U = np.asarray(U.todense()).T  # Convert to dense and transpose
        
        # Normalize U if required
        if donorm:
            U = U / np.linalg.norm(U, axis=0)
            U[np.isnan(U)] = 0  # Replace NaN values with zero
        
        # Solve for W
        if mode == 1:
            wf = (1 - lda2)  # Worst-fit tolerance for sparsity
            W = sparse_decode(X, U, lda1, worstFit=wf, mink=mink)
        elif mode == 2:
            if len(U0) > 0:
                W = projected_grad_desc(U, X, W, [], lda1, 0., nonneg=posW, maxItr=400)
            else:
                W = spams.lasso(np.asfortranarray(X), D=np.asfortranarray(U),
                                lambda1=lda1, lambda2=1.0, mode=2, numThreads=THREADS,
                                cholesky=use_chol, pos=posW)
                W = np.asarray(W.todense())  # Convert to dense
        
        # Compute updated reconstruction of X
        Xhat = U.dot(W)
        
        # Compute module and activity sizes based on entropy
        module_size = np.average([np.exp(entropy(abs(u))) for u in U.T if u.sum() > 0])
        activity_size = np.average([np.exp(entropy(abs(w))) for w in W.T])
        
        # Print progress if required
        if doprint:
            print(distance.correlation(X.flatten(), Xhat.flatten()), module_size, activity_size, lda1, lda2)
        
        # Adjust sparsity parameters dynamically
        if module_size < module_lower:
            lda2 /= 2.  # Decrease sparsity regularization for U
        if activity_size < activity_lower:
            lda2 /= 2.  # Decrease sparsity regularization for W
    
    return U, W  # Return learned matrices

## Random Double Balanced Matrix

In [9]:
def random_double_balanced(m, g, max_pools_per_gene, min_pools_per_gene):
    """
    Generates a random measurement matrix (`phi`) with "double balanced" characteristics.
    
    This function ensures:
    - Each gene is assigned to `max_pools_per_gene` pools initially.
    - Genes with fewer than `min_pools_per_gene` assignments are reassigned.
    - The resulting matrix is normalized so that each row sums to 1.

    Parameters:
    - m : int
        Number of pools (rows in the matrix).
    - g : int
        Number of genes (columns in the matrix).
    - max_pools_per_gene : int
        Maximum number of pools a gene can be assigned to.
    - min_pools_per_gene : int
        Minimum number of pools a gene must be assigned to.

    Returns:
    - phi : np.ndarray
        A (m × g) measurement matrix where each gene is assigned to a set of pools.
    """

    # Initialize an empty measurement matrix (pools × genes)
    phi = np.zeros((m, g))

    # Randomly assign each gene to pools up to `max_pools_per_gene` times
    for i in range(max_pools_per_gene):
        idx = np.random.choice(g, g, replace=False)  # Shuffle gene indices
        idx = idx % m  # Ensure indices fit within the pool size (modulo operation)
        phi[idx, np.arange(g)] = 1  # Assign genes to random pools

    # Ensure each gene is assigned to at least `min_pools_per_gene` pools
    for i in np.where(phi.sum(0) < min_pools_per_gene)[0]:  # Identify under-assigned genes
        p = phi.sum(1).max() - phi.sum(1)  # Compute imbalance for each pool
        p[np.where(phi[:, i])[0]] = 0  # Exclude pools that already contain the gene

        # If there are pools available for reassignment
        if p.sum() > 0:
            p = p / p.sum()  # Normalize probability distribution
            num_to_assign = min((p > 0).sum(), int(min_pools_per_gene - phi[:, i].sum()))
            idx = np.random.choice(m, num_to_assign, replace=False, p=p)  # Select new pools
            phi[idx, i] = 1  # Assign gene to additional pools

    # Normalize the matrix so that each row sums to 1 (avoid bias in pooling)
    phi = (phi.T / phi.sum(1)).T

    return phi

In [10]:
def find_best_coherence_matrices(m, g, max_pools_per_gene, min_pools_per_gene, U, num_matrices=5000, num_best=500):
    """
    Generates a set of random measurement matrices and selects the `num_best` matrices
    with the lowest (best) 90th percentile coherence scores.

    Parameters:
        num_matrices (int): Total number of random matrices to generate.
        num_best (int): Number of best matrices to retain.
        num_pools (int): Number of pools (rows in the measurement matrix).
        num_features (int): Number of features (columns in the measurement matrix).
        num_min (int): Minimum number of pools assigned per feature.
        num_max (int): Maximum number of pools assigned per feature.
        U (np.ndarray): Feature transformation matrix for coherence computation.

    Returns:
        tuple: (best coherence scores, corresponding best measurement matrices)
    """
    best = np.ones(num_best)  # Initialize best coherence scores (worst possible score = 1)
    Phi_coh = [None for _ in best]  # Corresponding best measurement matrices

    for x in range(num_matrices):  
        if np.mod(x, num_matrices // 10) == 0:  # Print progress every 10% of iterations
            print(f"Iteration {x} of {num_matrices}")

        # Generate a new random measurement matrix
        phi = random_double_balanced(m, g, max_pools_per_gene, min_pools_per_gene)  

        # Compute the 90th percentile of the cosine distance between projected feature vectors
        coh_90 = np.percentile(1 - distance.pdist(phi.dot(U).T, 'cosine'), 90)

        # If the new `phi` has a better coherence score, replace the worst-performing matrix
        if coh_90 < best.max():  
            i = np.argmax(best)  # Find the index of the worst current coherence score
            best[i] = coh_90  # Replace the worst score with the new, better coherence score
            Phi_coh[i] = phi  # Store the corresponding measurement matrix
    
    return best, Phi_coh

In [11]:
def find_best_reconstruction_matrices(Phi_coh, validate_data, U, sparsity=0.02, num_best=50):
    """
    Selects the best measurement matrices based on their ability to recover original gene expression patterns.

    Parameters:
        Phi_coh (list): List of best measurement matrices from coherence selection.
        validate_data (np.ndarray): Original validation dataset.
        U (np.ndarray): Feature transformation matrix.
        sparsity (float): Sparsity constraint for sparse decoding (default: 0.02).
        num_best (int): Number of best matrices to retain.

    Returns:
        tuple: (best reconstruction scores, corresponding best measurement matrices)
    """
    best = np.zeros(num_best)  # Initialize best reconstruction scores (worst possible score = 0)
    Phi = [None for _ in best]  # Corresponding best measurement matrices

    for phi in Phi_coh:
        # Generate simulated observations by applying `phi` to the validation data
        y = get_observations(validate_data, phi, snr=5)  # Simulate pooled noisy measurements
        
        # Use sparse decoding (LASSO) to recover gene module activations
        w = sparse_decode(y, phi.dot(U), sparsity, method='lasso')  
        
        # Reconstruct gene expression using the estimated module activations
        x2 = U.dot(w)  # Approximate gene expression matrix from recovered module weights
        
        # Compare reconstructed expression (`x2`) with the original validation data (`validate_data`)
        r = compare_results(validate_data, x2)  
        
        # If the new measurement matrix `phi` produces a better reconstruction, update `best` and `Phi`
        if r[2] > best.min():  # If the new reconstruction score is better than the worst stored score
            i = np.argmin(best)  # Find the index of the worst-performing matrix
            best[i] = r[2]  # Replace the worst reconstruction score with the new, better score
            Phi[i] = phi  # Store the corresponding measurement matrix
    
    return best, Phi

# Simulation

In [15]:
def run_simulation(
    adata_path, 
    gene_set_size,
    num_cells, 
    num_measurements, 
    min_pools_per_gene, 
    max_pools_per_gene, 
    sparsity, 
    num_modules,
    dataset_dir="./dataset/"
):
    """
    Runs the full simulation pipeline to optimize measurement matrices for gene expression analysis.
    
    Parameters:
        adata_path (str): Path to the AnnData file.
        gene_set_size (int): Number of genes to include in the simulation (500, 1000, or 5000).
        num_cells (int): Number of cells to include in the simulation.
        num_measurements (int): Number of measurement pools.
        min_pools_per_gene (int): Minimum pools a gene must be assigned to.
        max_pools_per_gene (int): Maximum pools a gene can be assigned to.
        sparsity (float): Sparsity constraint for sparse decoding.
        num_modules (int): Number of gene modules (dictionary size in SMAF).
        dataset_dir (str): Directory where the dataset is stored.
    
    Returns:
        dict: Best measurement matrix and evaluation metrics.
    """
    
    # Load the AnnData object
    adata = sc.read_h5ad(adata_path)
    
    # Downsample the AnnData object
    adata_subset = downsample_adata(adata, gene_set_size, num_cells, output_path=dataset_dir)
    
    # Split the dataset into train, validation, and test sets
    split_and_save_data(adata_subset, "subset_data", dataset_dir)
    
    # Load the split data
    train_data, validate_data, test_data = load_saved_data("subset_data", dataset_dir)
    
    # Generate gene modules matrix U using SMAF
    U, W = smaf(train_data, num_modules, lda1=8, lda2=0.2, maxItr=20,
                use_chol=False, donorm=True, mode=1, mink=0., doprint=True)
    
    # Remove zero-contribution modules
    nz = (U.sum(axis=0) > 0)
    U = U[:, nz]
    
    print("U dimentions =", U.shape)
    
    # Generate and evaluate measurement matrices based on coherence
    best_coh_scores, Phi_coh = find_best_coherence_matrices(m=num_measurements, g=train_data.shape[0], 
        min_pools_per_gene=min_pools_per_gene, max_pools_per_gene=max_pools_per_gene, U=U,
                                                            num_matrices=5000, num_best=500,
    )
    
    # Find best measurement matrices based on reconstruction quality
    best_rec_scores, Phi_best = find_best_reconstruction_matrices(
        Phi_coh, validate_data, U, sparsity, num_best=50
    )
    
    # Select the best matrix based on reconstruction quality
    best_idx = np.argmax(best_rec_scores)
    best_matrix = Phi_best[best_idx]
    best_score = best_rec_scores[best_idx]
    
    return {
        "best_matrix": best_matrix,
        "best_score": best_score,
        "best_coherence_scores": best_coh_scores,
        "best_reconstruction_scores": best_rec_scores
    }


# Runs

In [21]:
# Set parameters
adata_path = "dataset/cerebellum/cb_adult_mouse.h5ad"  # Path to your AnnData file
gene_set_size = 1000  # Choose from 500, 1000, or 5000
num_cells = 10000  # Number of cells to include in the simulation
num_measurements = 50  # Number of measurement pools
min_pools_per_gene = 4  # Minimum number of pools per gene
max_pools_per_gene = 4  # Maximum number of pools per gene
sparsity = 0.02  # Sparsity constraint for sparse decoding
num_modules = 50  # Number of gene modules
dataset_dir = "./dataset/cerebellum/cb_mouse"  # Directory where datasets are stored

# Run the simulation
results = run_simulation(
    adata_path=adata_path,
    gene_set_size=gene_set_size,
    num_cells=num_cells,
    num_measurements=num_measurements,
    min_pools_per_gene=min_pools_per_gene,
    max_pools_per_gene=max_pools_per_gene,
    sparsity=sparsity,
    num_modules=num_modules,
    dataset_dir=dataset_dir
)

# Print results
print("Best Measurement Matrix Shape:", results["best_matrix"].shape)
print("Best Score:", results["best_score"])
print("Top 5 Coherence Scores:", results["best_coherence_scores"][:5])
print("Top 5 Reconstruction Scores:", results["best_reconstruction_scores"][:5])

Matched genes for 1000 set: 892
Downsampled dataset saved to ./dataset/cerebellum/cb_mouse_1000genes.h5ad
Train data shape: (5000, 892)
Validation data shape: (2500, 892)
Test data shape: (2500, 892)
Datasets saved in ./dataset/cerebellum/cb_mouse as subset_data_train_data.npy, subset_data_validate_data.npy, and subset_data_test_data.npy
Loaded dataset: subset_data
Train data shape: (892, 5000)
Validation data shape: (892, 2500)
Test data shape: (892, 2500)
n_genes = 892
Iteration 0 of 5000
Iteration 500 of 5000
Iteration 1000 of 5000
Iteration 1500 of 5000
Iteration 2000 of 5000
Iteration 2500 of 5000
Iteration 3000 of 5000
Iteration 3500 of 5000
Iteration 4000 of 5000
Iteration 4500 of 5000


/projectnb/bioinfor/myousry/miniconda3/envs/comprsense/lib/python3.12/site-packages/scipy/spatial/distance.py:685: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


Best Measurement Matrix Shape: (50, 892)
Best Score: 0.6423503633716635
Top 5 Coherence Scores: [0.81664043 0.81957561 0.81482166 0.81603927 0.81343413]
Top 5 Reconstruction Scores: [0.6383733  0.63653413 0.63630183 0.63523376 0.63525676]


In [33]:
# Set parameters
adata_path = "dataset/cerebellum/cb_adult_mouse.h5ad"  # Path to your AnnData file
gene_set_size = 500  # Choose from 500, 1000, or 5000
num_cells = 10000  # Number of cells to include in the simulation
num_measurements = 50  # Number of measurement pools
min_pools_per_gene = 4  # Minimum number of pools per gene
max_pools_per_gene = 4  # Maximum number of pools per gene
sparsity = 0.02  # Sparsity constraint for sparse decoding
num_modules = 50  # Number of gene modules
dataset_dir = "./dataset/cerebellum/cb_mouse_500"  # Directory where datasets are stored

# Run the simulation
results = run_simulation(
    adata_path=adata_path,
    gene_set_size=gene_set_size,
    num_cells=num_cells,
    num_measurements=num_measurements,
    min_pools_per_gene=min_pools_per_gene,
    max_pools_per_gene=max_pools_per_gene,
    sparsity=sparsity,
    num_modules=num_modules,
    dataset_dir=dataset_dir
)

# Print results
print("Best Measurement Matrix Shape:", results["best_matrix"].shape)
print("Best Score:", results["best_score"])
print("Top 5 Coherence Scores:", results["best_coherence_scores"][:5])
print("Top 5 Reconstruction Scores:", results["best_reconstruction_scores"][:5])

Matched genes for 500 set: 444
Downsampled dataset saved to ./dataset/cerebellum/cb_mouse_500_500genes.h5ad
Train data shape: (5000, 444)
Validation data shape: (2500, 444)
Test data shape: (2500, 444)
Datasets saved in ./dataset/cerebellum/cb_mouse_500 as subset_data_train_data.npy, subset_data_validate_data.npy, and subset_data_test_data.npy
Loaded dataset: subset_data
Train data shape: (444, 5000)
Validation data shape: (444, 2500)
Test data shape: (444, 2500)
n_genes = 444
Iteration 0 of 5000
Iteration 500 of 5000
Iteration 1000 of 5000
Iteration 1500 of 5000
Iteration 2000 of 5000
Iteration 2500 of 5000
Iteration 3000 of 5000
Iteration 3500 of 5000
Iteration 4000 of 5000
Iteration 4500 of 5000


/projectnb/bioinfor/myousry/miniconda3/envs/comprsense/lib/python3.12/site-packages/scipy/spatial/distance.py:685: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


Best Measurement Matrix Shape: (50, 444)
Best Score: 0.3605855130943314
Top 5 Coherence Scores: [0.83021409 0.84573697 0.84569205 0.84427742 0.85104683]
Top 5 Reconstruction Scores: [0.35449873 0.35638015 0.35613819 0.35691696 0.35438521]


In [39]:
import numpy as np
import pandas as pd
import scanpy as sc  # For AnnData

def run_simulation(
    adata_path, 
    gene_set_size,
    num_cells, 
    num_measurements, 
    min_pools_per_gene, 
    max_pools_per_gene, 
    sparsity, 
    num_modules,
    dataset_dir="./dataset/",
    subset_gene_indices=None
):
    """
    Runs the full simulation pipeline to optimize measurement matrices for gene expression analysis.
    Now correctly extracts subset gene reconstruction scores.
    """
    
    # Load the AnnData object
    adata = sc.read_h5ad(adata_path)
    
    # Downsample the AnnData object
    adata_subset = downsample_adata(adata, gene_set_size, num_cells, output_path=dataset_dir)
    
    # Split the dataset into train, validation, and test sets
    split_and_save_data(adata_subset, "subset_data", dataset_dir)
    
    # Load the split data
    train_data, validate_data, test_data = load_saved_data("subset_data", dataset_dir)
    
    # Convert validation data to correct format (Fixes SPAMS error)
    validate_data = np.asfortranarray(validate_data.astype(np.float64))
    
    # Generate gene modules matrix U using SMAF
    U, W = smaf(train_data, num_modules, lda1=8, lda2=0.2, maxItr=100)
    
    # Remove zero-contribution modules
    nz = (U.sum(axis=0) > 0)
    U = U[:, nz]
    
    print("n_genes =", train_data.shape[0])
    
    # Generate and evaluate measurement matrices based on coherence
    best_coh_scores, Phi_coh = find_best_coherence_matrices(m=num_measurements, g=train_data.shape[0], 
        min_pools_per_gene=min_pools_per_gene, max_pools_per_gene=max_pools_per_gene, U=U,
        num_matrices=5000, num_best=500
    )
    
    # Find best measurement matrices based on reconstruction quality
    best_rec_scores_all, best_rec_scores_subset, Phi_best = find_best_reconstruction_matrices(
        Phi_coh, validate_data, U, sparsity, num_best=50, subset_gene_indices=subset_gene_indices
    )

    # Select the best matrix based on reconstruction quality
    best_idx = np.argmax(best_rec_scores_all)
    best_matrix = Phi_best[best_idx]
    best_score = best_rec_scores_all[best_idx]
    
    return {
        "best_matrix": best_matrix,
        "best_score": best_score,
        "best_coherence_scores": best_coh_scores,
        "best_reconstruction_scores_all": best_rec_scores_all,
        "best_reconstruction_scores_subset": best_rec_scores_subset
    }


def find_best_reconstruction_matrices(
    Phi_coh, validate_data, U, sparsity=0.02, num_best=50, subset_gene_indices=None
):
    """
    Selects the best measurement matrices based on their ability to recover original gene expression patterns.
    Now correctly extracts the third value from `compare_results()`, which is the reconstruction score.
    """
    best_all = np.zeros(num_best)  
    best_subset = np.zeros(num_best) if subset_gene_indices is not None else None
    Phi = [None for _ in best_all]  

    for i, phi in enumerate(Phi_coh):
        y = get_observations(validate_data, phi, snr=5)
        w = sparse_decode(y, phi.dot(U), sparsity, method='lasso')
        x2 = U.dot(w)  

        # Extract reconstruction score correctly
        r = compare_results(validate_data, x2)
        rec_score = r[2]  # Extract the reconstruction score

        subset_score = None
        if subset_gene_indices is not None:
            subset_reconstruction = x2[subset_gene_indices, :]
            subset_original = validate_data[subset_gene_indices, :]
            subset_score = compare_results(subset_original, subset_reconstruction)[2]  # Extract score

        if rec_score > best_all.min():
            idx = np.argmin(best_all)
            best_all[idx] = rec_score  
            Phi[idx] = phi  
            if subset_score is not None:
                best_subset[idx] = subset_score  

    return best_all, best_subset, Phi


# ========== Load Gene Lists and Identify Subset Indices ==========
genes_500 = pd.read_csv("./dataset/genes_500.csv", header=None, skiprows=1)[0].tolist()
genes_1000 = pd.read_csv("./dataset/genes_1000.csv", header=None, skiprows=1)[0].tolist()

# Find indices of genes_500 in genes_1000
indices_500_in_1000 = [i for i, gene in enumerate(genes_1000) if gene in genes_500]

# ========== Run Simulation for the 1000-Gene Set ==========
simulation_results = run_simulation(
    adata_path="./dataset/cerebellum/cb_mouse_1000genes.h5ad", 
    gene_set_size=1000,
    num_cells=10000, 
    num_measurements=50, 
    min_pools_per_gene=4, 
    max_pools_per_gene=4, 
    sparsity=0.02, 
    num_modules=50,
    dataset_dir="./dataset/cerebellum",
    subset_gene_indices=indices_500_in_1000  # Pass the subset indices
)

# Extract reconstruction scores
best_rec_scores_all = simulation_results["best_reconstruction_scores_all"]
best_rec_scores_subset = simulation_results["best_reconstruction_scores_subset"]

# Compute means
mean_full_score = np.mean(best_rec_scores_all)
mean_subset_score = np.mean(best_rec_scores_subset) if best_rec_scores_subset is not None else None

# ========== Print Results ==========
print(f"Mean recovery score for all 1000 genes: {mean_full_score:.4f}")
if mean_subset_score is not None:
    print(f"Mean recovery score for the 500 genes (from 1000-gene simulation): {mean_subset_score:.4f}")

Matched genes for 1000 set: 892
Downsampled dataset saved to ./dataset/cerebellum_1000genes.h5ad
Train data shape: (5000, 892)
Validation data shape: (2500, 892)
Test data shape: (2500, 892)
Datasets saved in ./dataset/cerebellum as subset_data_train_data.npy, subset_data_validate_data.npy, and subset_data_test_data.npy
Loaded dataset: subset_data
Train data shape: (892, 5000)
Validation data shape: (892, 2500)
Test data shape: (892, 2500)
n_genes = 892
Iteration 0 of 5000
Iteration 500 of 5000
Iteration 1000 of 5000
Iteration 1500 of 5000
Iteration 2000 of 5000
Iteration 2500 of 5000
Iteration 3000 of 5000
Iteration 3500 of 5000
Iteration 4000 of 5000
Iteration 4500 of 5000


/projectnb/bioinfor/myousry/miniconda3/envs/comprsense/lib/python3.12/site-packages/scipy/spatial/distance.py:685: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


Mean recovery score for all 1000 genes: 0.5261
Mean recovery score for the 500 genes (from 1000-gene simulation): 0.5126


In [16]:
import numpy as np
import pandas as pd
import scanpy as sc  # For AnnData

def run_simulation(
    adata_path, 
    gene_set_size,
    num_cells, 
    num_measurements, 
    min_pools_per_gene, 
    max_pools_per_gene, 
    sparsity, 
    num_modules,
    dataset_dir="./dataset/",
    subset_gene_indices=None
):
    """
    Runs the full simulation pipeline to optimize measurement matrices for gene expression analysis.
    Now correctly extracts subset gene reconstruction scores.
    """
    
    # Load the AnnData object
    adata = sc.read_h5ad(adata_path)
    
    # Downsample the AnnData object
    adata_subset = downsample_adata(adata, gene_set_size, num_cells, output_path=dataset_dir)
    
    # Split the dataset into train, validation, and test sets
    split_and_save_data(adata_subset, "subset_data", dataset_dir)
    
    # Load the split data
    train_data, validate_data, test_data = load_saved_data("subset_data", dataset_dir)
    
    # Convert validation data to correct format (Fixes SPAMS error)
    validate_data = np.asfortranarray(validate_data.astype(np.float64))
    
    # Generate gene modules matrix U using SMAF
    U, W = smaf(train_data, num_modules, lda1=8, lda2=0.2, maxItr=100)
    
    # Remove zero-contribution modules
    nz = (U.sum(axis=0) > 0)
    U = U[:, nz]
    
    print("n_genes =", train_data.shape[0])
    
    # Generate and evaluate measurement matrices based on coherence
    best_coh_scores, Phi_coh = find_best_coherence_matrices(m=num_measurements, g=train_data.shape[0], 
        min_pools_per_gene=min_pools_per_gene, max_pools_per_gene=max_pools_per_gene, U=U,
        num_matrices=5000, num_best=500
    )
    
    # Find best measurement matrices based on reconstruction quality
    best_rec_scores_all, best_rec_scores_subset, Phi_best = find_best_reconstruction_matrices(
        Phi_coh, validate_data, U, sparsity, num_best=50, subset_gene_indices=subset_gene_indices
    )

    # Select the best matrix based on reconstruction quality
    best_idx = np.argmax(best_rec_scores_all)
    best_matrix = Phi_best[best_idx]
    best_score = best_rec_scores_all[best_idx]
    
    return {
        "best_matrix": best_matrix,
        "best_score": best_score,
        "best_coherence_scores": best_coh_scores,
        "best_reconstruction_scores_all": best_rec_scores_all,
        "best_reconstruction_scores_subset": best_rec_scores_subset
    }


def find_best_reconstruction_matrices(
    Phi_coh, validate_data, U, sparsity=0.02, num_best=50, subset_gene_indices=None
):
    """
    Selects the best measurement matrices based on their ability to recover original gene expression patterns.
    Now correctly extracts the third value from `compare_results()`, which is the reconstruction score.
    """
    best_all = np.zeros(num_best)  
    best_subset = np.zeros(num_best) if subset_gene_indices is not None else None
    Phi = [None for _ in best_all]  

    for i, phi in enumerate(Phi_coh):
        y = get_observations(validate_data, phi, snr=5)
        w = sparse_decode(y, phi.dot(U), sparsity, method='lasso')
        x2 = U.dot(w)  

        # Extract reconstruction score correctly
        r = compare_results(validate_data, x2)
        rec_score = r[2]  # Extract the reconstruction score

        subset_score = None
        if subset_gene_indices is not None:
            subset_reconstruction = x2[subset_gene_indices, :]
            subset_original = validate_data[subset_gene_indices, :]
            subset_score = compare_results(subset_original, subset_reconstruction)[2]  # Extract score

        if rec_score > best_all.min():
            idx = np.argmin(best_all)
            best_all[idx] = rec_score  
            Phi[idx] = phi  
            if subset_score is not None:
                best_subset[idx] = subset_score  

    return best_all, best_subset, Phi


# ========== Load Gene Lists and Identify Subset Indices ==========
genes_500 = pd.read_csv("./dataset/genes_500.csv", header=None, skiprows=1)[0].tolist()
genes_1000 = pd.read_csv("./dataset/genes_1000.csv", header=None, skiprows=1)[0].tolist()

# Find indices of genes_500 in genes_1000
indices_500_in_1000 = [i for i, gene in enumerate(genes_1000) if gene in genes_500]

# ========== Run Simulation for the 1000-Gene Set ==========
simulation_results = run_simulation(
    adata_path="./dataset/cerebellum/cb_mouse_1000genes.h5ad", 
    gene_set_size=1000,
    num_cells=10000, 
    num_measurements=50, 
    min_pools_per_gene=4, 
    max_pools_per_gene=4, 
    sparsity=0.02, 
    num_modules=50,
    dataset_dir="./dataset/cerebellum",
    subset_gene_indices=indices_500_in_1000  # Pass the subset indices
)

# Extract reconstruction scores
best_rec_scores_all = simulation_results["best_reconstruction_scores_all"]
best_rec_scores_subset = simulation_results["best_reconstruction_scores_subset"]

# Compute means
mean_full_score = np.mean(best_rec_scores_all)
mean_subset_score = np.mean(best_rec_scores_subset) if best_rec_scores_subset is not None else None

# ========== Print Results ==========
print(f"Mean recovery score for all 1000 genes: {mean_full_score:.4f}")
if mean_subset_score is not None:
    print(f"Mean recovery score for the 500 genes (from 1000-gene simulation): {mean_subset_score:.4f}")

Matched genes for 1000 set: 892
Downsampled dataset saved to ./dataset/cerebellum_1000genes.h5ad
Train data shape: (5000, 892)
Validation data shape: (2500, 892)
Test data shape: (2500, 892)
Datasets saved in ./dataset/cerebellum as subset_data_train_data.npy, subset_data_validate_data.npy, and subset_data_test_data.npy
Loaded dataset: subset_data
Train data shape: (892, 5000)
Validation data shape: (892, 2500)
Test data shape: (892, 2500)
n_genes = 892
Iteration 0 of 5000
Iteration 500 of 5000
Iteration 1000 of 5000
Iteration 1500 of 5000
Iteration 2000 of 5000
Iteration 2500 of 5000
Iteration 3000 of 5000
Iteration 3500 of 5000
Iteration 4000 of 5000
Iteration 4500 of 5000


/projectnb/bioinfor/myousry/miniconda3/envs/comprsense/lib/python3.12/site-packages/scipy/spatial/distance.py:685: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


Mean recovery score for all 1000 genes: 0.6124
Mean recovery score for the 500 genes (from 1000-gene simulation): 0.6318


In [13]:
# Set parameters
adata_path = "dataset/cerebellum/cb_adult_mouse.h5ad"  # Path to your AnnData file
gene_set_size = 1000  # Choose from 500, 1000, or 5000
num_cells = 100000  # Number of cells to include in the simulation
num_measurements = 50  # Number of measurement pools
min_pools_per_gene = 4  # Minimum number of pools per gene
max_pools_per_gene = 4  # Maximum number of pools per gene
sparsity = 0.02  # Sparsity constraint for sparse decoding
num_modules = 50  # Number of gene modules
dataset_dir = "./dataset/cerebellum/cb_mouse_1000"  # Directory where datasets are stored

# Run the simulation
results = run_simulation(
    adata_path=adata_path,
    gene_set_size=gene_set_size,
    num_cells=num_cells,
    num_measurements=num_measurements,
    min_pools_per_gene=min_pools_per_gene,
    max_pools_per_gene=max_pools_per_gene,
    sparsity=sparsity,
    num_modules=num_modules,
    dataset_dir=dataset_dir
)

# Print results
print("Best Measurement Matrix Shape:", results["best_matrix"].shape)
print("Best Score:", results["best_score"])
print("Top 5 Coherence Scores:", results["best_coherence_scores"][:5])
print("Top 5 Reconstruction Scores:", results["best_reconstruction_scores"][:5])

Matched genes for 1000 set: 892
Downsampled dataset saved to ./dataset/cerebellum/cb_mouse_1000_1000genes.h5ad
Train data shape: (50000, 892)
Validation data shape: (25000, 892)
Test data shape: (25000, 892)
Datasets saved in ./dataset/cerebellum/cb_mouse_1000 as subset_data_train_data.npy, subset_data_validate_data.npy, and subset_data_test_data.npy
Loaded dataset: subset_data
Train data shape: (892, 50000)
Validation data shape: (892, 25000)
Test data shape: (892, 25000)
n_genes = 892
Iteration 0 of 5000
Iteration 500 of 5000
Iteration 1000 of 5000
Iteration 1500 of 5000
Iteration 2000 of 5000
Iteration 2500 of 5000
Iteration 3000 of 5000
Iteration 3500 of 5000
Iteration 4000 of 5000
Iteration 4500 of 5000


/projectnb/bioinfor/myousry/miniconda3/envs/comprsense/lib/python3.12/site-packages/scipy/spatial/distance.py:685: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


Best Measurement Matrix Shape: (50, 892)
Best Score: 0.3048025086010231
Top 5 Coherence Scores: [0.1306926  0.12800974 0.12828587 0.12685996 0.11526894]
Top 5 Reconstruction Scores: [0.30063431 0.30315967 0.29975268 0.29953545 0.29985891]


In [14]:
print('done')

done
